In [ ]:
import datetime
# import pandas_ta as ta
import pandas as pd

from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG

In [ ]:
import datetime
import pandas_ta as ta
import pandas as pd

from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG

class RsiOscillator(Strategy):

    upper_bound = 70
    lower_bound = 30
    rsi_window = 14

    # Do as much initial computation as possible
    def init(self):
        self.rsi = self.I(ta.rsi, pd.Series(self.data.Close), self.rsi_window)

    # Step through bars one by one
    # Note that multiple buys are a thing here
    def next(self):
        if crossover(self.rsi, self.upper_bound):
            self.position.close()
        elif crossover(self.lower_bound, self.rsi):
            self.buy()

bt = Backtest(GOOG, RsiOscillator, cash=10_000, commission=.002)
stats = bt.run()
bt.plot()


In [ ]:
print(GOOG)

In [ ]:
import pandas as pd

# Load the data from the uploaded CSV file
file_path = '../data/binance_btc_usdt_candlestick.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
data.head()


In [ ]:
data.describe()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.plot(data['timestamp'], data['close'], label='Closing Price', color='blue')
plt.title('BTC/USDT Closing Prices Over Time')
plt.xlabel('Timestamp')
#make the timestamp writing diagonal
plt.xticks(rotation=45)
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(data['timestamp'], data['volume'], label='Trading Volume', color='orange')
plt.title('BTC/USDT Trading Volume Over Time')
plt.xlabel('Timestamp')
plt.ylabel('Volume')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
data['20_day_MA'] = data['close'].rolling(window=20).mean()
data['50_day_MA'] = data['close'].rolling(window=50).mean()
data['15_day_MA'] = data['close'].rolling(window=15).mean()

In [ ]:
# Plot Moving Averages
plt.figure(figsize=(14, 7))
plt.plot(data['timestamp'], data['close'], label='Closing Price', color='blue')
plt.plot(data['timestamp'], data['20_day_MA'], label='20-Day MA', color='orange')
plt.plot(data['timestamp'], data['50_day_MA'], label='50-Day MA', color='green')
plt.plot(data['timestamp'], data['15_day_MA'], label='15-Day MA', color='red')
plt.title('BTC/USDT Moving Averages')
plt.xlabel('Timestamp')
plt.xticks(rotation=45)
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
data['Buy_Signal'] = (data['20_day_MA'] > data['15_day_MA']) & (data['20_day_MA'].shift(1) <= data['15_day_MA'].shift(1))
data['Sell_Signal'] = (data['20_day_MA'] < data['15_day_MA']) & (data['20_day_MA'].shift(1) >= data['15_day_MA'].shift(1))

# Plot Buy/Sell Signals
plt.figure(figsize=(14, 7))
plt.plot(data['timestamp'], data['close'], label='Closing Price', color='blue')
plt.plot(data['timestamp'], data['20_day_MA'], label='20-Day MA', color='orange')
plt.plot(data['timestamp'], data['50_day_MA'], label='50-Day MA', color='green')
plt.scatter(data.loc[data['Buy_Signal'], 'timestamp'], data.loc[data['Buy_Signal'], 'close'], label='Buy Signal', marker='^', color='green', s=100)
plt.scatter(data.loc[data['Sell_Signal'], 'timestamp'], data.loc[data['Sell_Signal'], 'close'], label='Sell Signal', marker='v', color='red', s=100)
plt.title('BTC/USDT Buy and Sell Signals')
plt.xlabel('Timestamp')
plt.ylabel('Price')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
data.loc[data['Buy_Signal']]

In [ ]:
data.loc[data['Sell_Signal']]

In [ ]:
# Separate the rows with date strings and millisecond timestamps
date_format_rows = data[~data['timestamp'].str.isnumeric()]
ms_format_rows = data[data['timestamp'].str.isnumeric()]

# Convert date strings to datetime
date_format_rows['timestamp'] = pd.to_datetime(date_format_rows['timestamp'])

# Convert millisecond timestamps to datetime
ms_format_rows['timestamp'] = pd.to_datetime(ms_format_rows['timestamp'].astype('int64'), unit='ms')

# Combine the two datasets
cleaned_data = pd.concat([date_format_rows, ms_format_rows])

# Sort the data by timestamp
cleaned_data = cleaned_data.sort_values(by='timestamp')

# Calculate 20-day and 50-day moving averages
cleaned_data['20_day_MA'] = cleaned_data['close'].rolling(window=20).mean()
cleaned_data['50_day_MA'] = cleaned_data['close'].rolling(window=50).mean()

# Identify buy and sell signals
cleaned_data['Buy_Signal'] = (cleaned_data['20_day_MA'] > cleaned_data['50_day_MA']) & (cleaned_data['20_day_MA'].shift(1) <= cleaned_data['50_day_MA'].shift(1))
cleaned_data['Sell_Signal'] = (cleaned_data['20_day_MA'] < cleaned_data['50_day_MA']) & (cleaned_data['20_day_MA'].shift(1) >= cleaned_data['50_day_MA'].shift(1))

# Plot Closing Prices and Moving Averages with Buy/Sell Signals
plt.figure(figsize=(14, 7))
plt.plot(cleaned_data['timestamp'], cleaned_data['close'], label='Closing Price', color='blue')
plt.plot(cleaned_data['timestamp'], cleaned_data['20_day_MA'], label='20-Day MA', color='orange')
plt.plot(cleaned_data['timestamp'], cleaned_data['50_day_MA'], label='50-Day MA', color='green')
plt.scatter(cleaned_data.loc[cleaned_data['Buy_Signal'], 'timestamp'], cleaned_data.loc[cleaned_data['Buy_Signal'], 'close'], label='Buy Signal', marker='^', color='green', s=100)
plt.scatter(cleaned_data.loc[cleaned_data['Sell_Signal'], 'timestamp'], cleaned_data.loc[cleaned_data['Sell_Signal'], 'close'], label='Sell Signal', marker='v', color='red', s=100)
plt.title('BTC/USDT Closing Prices and Moving Averages with Buy/Sell Signals')
plt.xlabel('Timestamp')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()

# Plot Trading Volume
plt.figure(figsize=(14, 7))
plt.plot(cleaned_data['timestamp'], cleaned_data['volume'], label='Trading Volume', color='orange')
plt.title('BTC/USDT Trading Volume Over Time')
plt.xlabel('Timestamp')
plt.ylabel('Volume')
plt.legend()
plt.grid(True)
plt.show()
# Display summary statistics
summary_stats = cleaned_data.describe()
summary_stats

In [ ]:
# Initialize variables
position = None
buy_date = None
buy_price = None

# Iterate through the data to track buy and sell actions
for index, row in cleaned_data.iterrows():
    if row['Buy_Signal'] and position is None:
        # Buy action
        position = 'Bought'
        buy_date = row['timestamp']
        buy_price = row['close']
    elif row['Sell_Signal'] and position == 'Bought':
        # Sell action
        position = None
        sell_date = row['timestamp']
        sell_price = row['close']
        profit = sell_price - buy_price
        trades = trades.append({'Buy Date': buy_date, 'Buy Price': buy_price, 'Sell Date': sell_date, 'Sell Price': sell_price, 'Profit': profit}, ignore_index=True)

# Calculate total profit
total_profit = trades['Profit'].sum()
total_profit

In [ ]:
# Create a DataFrame to keep track of buy and sell actions
trades = pd.DataFrame(columns=['Buy Date', 'Buy Price', 'Sell Date', 'Sell Price', 'Profit'])

# Initialize variables
position = None
buy_date = None
buy_price = None

# List to collect trade data
trade_data = []

# Iterate through the data to track buy and sell actions
for index, row in cleaned_data.iterrows():
    if row['Buy_Signal'] and position is None:
        # Buy action
        position = 'Bought'
        buy_date = row['timestamp']
        buy_price = row['close']
    elif row['Sell_Signal'] and position == 'Bought':
        # Sell action
        position = None
        sell_date = row['timestamp']
        sell_price = row['close']
        profit = sell_price - buy_price
        trade_data.append({'Buy Date': buy_date, 'Buy Price': buy_price, 'Sell Date': sell_date, 'Sell Price': sell_price, 'Profit': profit})

# Convert the list of trade data to a DataFrame
trades = pd.DataFrame(trade_data)

# Calculate total profit
total_profit = trades['Profit'].sum()
total_profit


In [ ]:
trades

In [1]:
from sqlalchemy import create_engine, Table, MetaData
import os

# Create engine and connect to the database
engine = create_engine(os.getenv('PYCOPG_DATABASE_URL'))

In [2]:
connection = engine.connect()
metadata = MetaData()

In [ ]:
# Reflect the 'scenes' table
scenes = Table('scenes', metadata, autoload_with=engine)

# Build the query
query = scenes.select().limit(10).offset(0)

# Execute the query
result = connection.execute(query)
for row in result:
    print(row)

In [4]:
from sqlalchemy import create_engine, text, inspect
from sqlalchemy.orm import sessionmaker

# Create an engine and connect to the database
Session = sessionmaker(bind=engine)
session = Session()



In [9]:

# Execute the DROP TABLE command with CASCADE
session.execute(text("DROP TABLE stocks CASCADE"))
session.commit()

# Close the session
session.close()

In [11]:
# Close the session
session.close()

In [10]:
# Create an inspector object
inspector = inspect(engine)

# Get the list of all tables
tables = inspector.get_table_names()

# Print the list of tables
print(tables)

[]


In [ ]:
import backtrader as bt
import pandas as pd

class SMAStrategy(bt.Strategy):
    params = (
        ('period', 15),
    )

    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.period)
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.trade_count = 0
        self.winning_trades = 0
        self.losing_trades = 0
        self.max_drawdown = 0

    def next(self):
        if self.order:
            return

        if self.sma > self.data.close:
            self.order = self.buy()
        elif self.sma < self.data.close:
            self.order = self.sell()

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.sellprice = order.executed.price
                self.sellcomm = order.executed.comm

            self.bar_executed = len(self)

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        print(f'Operation Profit, Gross {trade.pnl}, Net {trade.pnlcomm}')
        self.trade_count += 1
        if trade.pnl > 0:
            self.winning_trades += 1
        else:
            self.losing_trades += 1

        self.max_drawdown = max(self.max_drawdown, self.broker.get_value() - self.data.close[0])
        sharpe_ratio = self.broker.get_value() / self.broker.get_cash()

        self.cerebro.metrics.append({
            'Period': self.params.period,
            'Gross Profit': trade.pnl,
            'Net Profit': trade.pnlcomm,
            'Number of Trades': self.trade_count,
            'Winning Trades': self.winning_trades,
            'Losing Trades': self.losing_trades,
            'Max Drawdown': self.max_drawdown,
            'Sharpe Ratio': sharpe_ratio
        })


In [ ]:
df = pd.read_csv('../data/binance_btc_usdt_candlestick.csv', index_col='timestamp', parse_dates=True)

In [ ]:
def run_backtest(scene, df):
    cerebro = bt.Cerebro()
    data = bt.feeds.PandasData(dataname=df)
    cerebro.adddata(data)

    if scene['indicator_name'] == 'SMA':
        cerebro.addstrategy(SMAStrategy, period=scene['period'])
    # Add more strategies as needed

    cerebro.metrics = []
    cerebro.run()

    return cerebro.metrics

In [ ]:
run_backtest({"indicator_name": "SMA", "period": 20}, df)